In [ ]:
import os
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from einops import rearrange
from gluonts.dataset.pandas import PandasDataset, is_uniform, infer_freq
from gluonts.dataset.split import split
from uni2ts.model.moirai import MoiraiForecast, MoiraiModule
from uni2ts.eval_util.plot import plot_single, plot_next_multi
from uni2ts.eval_util.evaluation import evaluate_model
from gluonts.ev.metrics import MAE, MAPE

from mov_av import moving_average

In [ ]:
it_data_23_24 = pd.read_csv('data/it_load_data_23_24.csv', delimiter=',')
it_data_23_24['DateUTC'] = pd.to_datetime(it_data_23_24['DateUTC'])
it_data_23_24 = it_data_23_24.set_index('DateUTC')

In [ ]:
# PDT = 7 * 24  # prediction length: any positive integer
# PSZ = "auto"  # patch size: choose from {"auto", 8, 16, 32, 64, 128}
# BSZ = 24  # batch size: any positive integer

# DIST = PDT

In [ ]:
from uni2ts.alex.common.benchmark_func import *

In [ ]:
data = it_data_23_24

# model_folder = "outputs/finetune/moirai_small/1st_run/checkpoints/it_load_data_train_"
# backtesting_with_refit = get_run_metrics(model_folder, data, "MAPE")

# model_folder = "outputs/finetune/moirai_small/2nd_run/checkpoints/it_load_data_train_"
# refit_1_pass =  get_run_metrics(model_folder, data, "MAPE")

# model_folder = "outputs/finetune/moirai_small/3rd_run/checkpoints/it_load_data_train_"
# refit_multiple_passes = get_run_metrics(model_folder, data, "MAPE")

model_folder = "pretrained"
pretrained = get_run_metrics(model_folder, prediction_lenght=7*24,
                             data=data, metric="MAPE", quantiles=[0.05, 0.5, 0.95], num_samples=20)

In [ ]:
backtesting_with_refit = pd.concat(backtesting_with_refit, axis=0)
refit_1_pass = pd.concat(refit_1_pass, axis=0)
refit_multiple_passes = pd.concat(refit_multiple_passes, axis=0)
pretrained = pd.concat(pretrained, axis=0)

In [ ]:
backtesting_with_refit['MAPE[0.95]'].plot(label='Backtesting with refit')
refit_1_pass['MAPE[0.95]'].plot(label='Refit (1 backword pass)')
refit_multiple_passes['MAPE[0.95]'].plot(label='Refit (multiple backword passes)')
pretrained['MAPE[0.95]'].plot(label='Pretrained')
plt.xticks(range(0, 26), range(1, 27))
plt.xlabel('Number of weeks')
plt.ylabel('MAPE')
plt.title('Benchmark on MAPE based on 95th percentile')
plt.legend()
plt.tight_layout()
plt.savefig("experiment_results/after_finetuning/mape_95th_perc.png");

In [ ]:
backtesting_with_refit['MAPE[0.5]'].plot(label='Backtesting with refit')
refit_1_pass['MAPE[0.5]'].plot(label='Refit (1 backword pass)')
refit_multiple_passes['MAPE[0.5]'].plot(label='Refit (multiple backword passes)')
pretrained['MAPE[0.5]'].plot(label='Pretrained')
plt.xticks(range(0, 26), range(1, 27))
plt.xlabel('Number of weeks')
plt.ylabel('MAPE')
plt.title('Benchmark on MAPE based on 50th percentile')
plt.legend()
plt.tight_layout()
plt.savefig("experiment_results/after_finetuning/mape_50th_perc.png");

In [ ]:
model_folder = "outputs/finetune/moirai_small/1st_run/checkpoints/it_load_data_train_"
data = it_data_23_24

model, test_data = get_model_data(model_folder, 26, data)

predictor = model.create_predictor(batch_size=BSZ)
forecasts = predictor.predict(test_data.input)

input_it = iter(test_data.input)
label_it = iter(test_data.label)
forecast_it = iter(forecasts)

NUM_WINDOWS = 1

fig, axes = plt.subplots(nrows=NUM_WINDOWS, ncols=1, figsize=(8, NUM_WINDOWS * 4))
plot_next_multi(
    axes,
    input_it,
    label_it,
    forecast_it,
    context_length=1 * 168,
    intervals=(0.9, 0.95),
    dim=None,
    name="Prediction",
    show_label=True,
)
plt.tight_layout()

In [ ]:
model = load_pretrained(720)

predictor = model.create_predictor(batch_size=BSZ)
forecasts = predictor.predict(test_data.input)

input_it = iter(test_data.input)
label_it = iter(test_data.label)
forecast_it = iter(forecasts)

NUM_WINDOWS = 1

fig, axes = plt.subplots(nrows=NUM_WINDOWS, ncols=1, figsize=(8, NUM_WINDOWS * 4))
plot_next_multi(
    axes,
    input_it,
    label_it,
    forecast_it,
    context_length=1 * 168,
    intervals=(0.9, 0.95),
    dim=None,
    name="Prediction",
    show_label=True,
)
plt.tight_layout()